tous les imports

In [1]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split    
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz
import sys
from joblib import load
import numpy as np
from joblib import dump

Question 1 : 

In [3]:

def comestible(url):
    reponse = requests.get(url)
    if reponse.status_code == 200:
        soup = BeautifulSoup(reponse.content, 'html.parser')
        div = soup.find('div', class_='cat_link').text.strip()
        if div == 'Poisonous Mushrooms':
            return 'P'
        elif div == 'Edible Mushrooms':
            return 'E'
        elif div == 'Inedible Mushrooms':
            return 'I'
        else:
            return'structure de page inconnue'
    return 'requete non reussie'


Question 2 : 

In [4]:

def color(url):
    try:
        reponse = requests.get(url)
        if reponse.status_code == 200:
            soup = BeautifulSoup(reponse.content, 'html.parser')
            div = soup.find('div', class_='mprofile')
            if div:
                couleur = div.find(lambda tag: tag.name == 'p' and 'Color:' in tag.text)
                if couleur:
                    liens = couleur.find_all('a')
                    if liens:  
                        couleurs = [link.get_text(strip=True) for link in liens]
                        return '-'.join(couleurs)
    except Exception as e:
        return ""


Question 3 : 

In [5]:

def shape(url):
    reponse = requests.get(url)
    if reponse.status_code == 200:
        soup = BeautifulSoup(reponse.content, 'html.parser')
        div = soup.find('div', class_='mprofile')
        if div:
            shape_str = div.find('strong', string='Shape:')
            if shape_str:
                shape = shape_str.find_next('a').text.strip().replace(' ', '')
                return shape
    return ''


def surface(url):
    reponse = requests.get(url)
    if reponse.status_code == 200:
        soup = BeautifulSoup(reponse.content, 'html.parser')
        div = soup.find('div', class_='mprofile')
        if div:
            surface_tag = div.find('strong', string='Surface:')
            if surface_tag:
                surface = surface_tag.find_next('a').text.strip()
                return surface
    return ''

Question 4 : 

In [6]:

def csvv(url):
    type = comestible(url)
    color_info = color(url)
    shape_info = shape(url)
    surface_info = surface(url)
    # Formater la chaîne CSV
    csv_info = f"{type},{color_info},{shape_info},{surface_info}"
    return csv_info

Main pour les 4 premiers questions :

In [7]:

url_poisonous = "https://ultimate-mushroom.com/poisonous/103-abortiporus-biennis.html"
url_edible = "https://ultimate-mushroom.com/edible/1010-agaricus-albolutescens.html"
url_inedible = "https://ultimate-mushroom.com/inedible/452-byssonectria-terrestris.html"


resultat_poisonous = comestible(url_poisonous)
resultat_edible = comestible(url_edible)
resultat_inedible = comestible(url_inedible)

print("Poisonous:", resultat_poisonous)
print("Edible:", resultat_edible)
print("Inedible:", resultat_inedible)

resultat_color_url1 = color(url_poisonous)
resultat_color_url2=  color(url_edible)  
resultat_color_url3 = color(url_inedible)
print("Color:", resultat_color_url1)
print("Color:", resultat_color_url2)
print("Color:", resultat_color_url3)

resultat_shape_url1 = shape(url_poisonous) 
resultat_shape_url2 = shape(url_edible)
resultat_shape_url3 = shape(url_inedible)
print("Shape:", resultat_shape_url1)
print("Shape:", resultat_shape_url2)
print("Shape:", resultat_shape_url3)

resultat_surface_url1 = surface(url_poisonous)
resultat_surface_url2 = surface(url_edible)
resultat_surface_url3 = surface(url_inedible)
print("Surface:", resultat_surface_url1)
print("Surface:", resultat_surface_url2)
print("Surface:", resultat_surface_url3)

resultat_csv_url1 = csvv(url_poisonous)
resultat_csv_url2 = csvv(url_edible)
resultat_csv_url3 = csvv(url_inedible)
print("csv:", resultat_csv_url1)
print("csv:", resultat_csv_url2)
print("csv:", resultat_csv_url3)


Poisonous: P
Edible: E
Inedible: I
Color: Pale
Color: White-Yellow
Color: Orange-Yellow
Shape: Polypore
Shape: Convex
Shape: CupFungi
Surface: Smooth
Surface: Smooth
Surface: Smooth
csv: P,Pale,Polypore,Smooth
csv: E,White-Yellow,Convex,Smooth
csv: I,Orange-Yellow,CupFungi,Smooth


Question 5 :
tout d'abord la fonction pour récupérer les liens de champignons
ensuite une fonction pour écrire dans le fichier CSV
puis la fonction principale pour parcourir les pages et générer le fichier CSV

In [7]:


def liensChampignion(): 
    x = requests.get("https://ultimate-mushroom.com/mushroom-alphabet.html")
    soup = BeautifulSoup(x.text, "html.parser")
    liens = []
    # Trouver tous les liens vers les pages de champignons
    for a in soup.find_all('a', href=True):
        href = a.get('href', '')
        if 'ultimate-mushroom.com/' in href:
            liens.append(href)


    return liens

def ecritureCsv(source, x):
    with open(source,'a', newline='', encoding='utf-8') as f:
        ecriture = csv.writer(f)
        ecriture.writerow(x)


def csvChampignions():
    liens = liensChampignion()
    ligne = ["Edible", "Color", "Shape", "Surface"]
    ecritureCsv("champignons.csv", ligne)

    for lien in liens:
        champignion_type = comestible(lien)
        champignion_color = color(lien)
        champignion_shape = shape(lien)
        champignion_surface = surface(lien)
        champignion_data = [champignion_type,champignion_color, champignion_shape, champignion_surface]
        ecritureCsv("champignons.csv", champignion_data)

In [28]:
#Appel a la fonction de creation de csv
# csvChampigions()

Question 7 : vérification de donnée ; par une importation de nos données ecrites dans un fichier csv 

In [29]:
champignons = pd.read_csv("champignons.csv")
print("Dimensions du DataFrame :", champignons.shape)
print("Noms des colonnes :", champignons.columns)

Dimensions du DataFrame : (210, 4)
Noms des colonnes : Index(['Edible', 'Color', 'Shape', 'Surface'], dtype='object')


Question 8 : Inspecter les données de la colonne "Edible"

Question 9 : ensuite remplacer les valeurs 'E', 'I' et 'P' par les valeurs numériques 0, 1 et 2 


In [30]:
print(champignons['Edible'].value_counts(dropna=False))
champignons['Edible'] = champignons['Edible'].replace({'E': 0, 'I': 1, 'P': 2})
print(champignons['Edible'].value_counts(dropna=False))
champignons.head()

Edible
E      100
P       81
I       27
NaN      2
Name: count, dtype: int64
Edible
0.0    100
2.0     81
1.0     27
NaN      2
Name: count, dtype: int64


Edible         Color        Shape      Surface
0     2.0          Pale     Polypore       Smooth
1     0.0         White       Convex       Smooth
2     0.0  White-Yellow       Convex       Smooth
3     0.0         White  Bell-shaped       Smooth
4     0.0         Brown  Bell-shaped  Flat Scales

Question 10 : À l’aide de la méthode fillna(), remplacer les valeurs manquantes par des -1.

In [10]:
champignons['Edible'] = champignons['Edible'].fillna(-1)
print(champignons['Edible'].value_counts(dropna=False))

Edible
 0.0    100
 2.0     81
 1.0     27
-1.0      2
Name: count, dtype: int64


Question 12 : Pour chaque valeur unique, créez une nouvelle colonne et remplissez-la avec 1 si la valeur apparaît dans le champ "Shape" de la ligne respective


In [14]:
formes = pd.unique(champignons['Shape'].str.split("-").explode().dropna())
for forme in formes:
    champignons[forme] = champignons['Shape'].str.contains(forme).fillna(False).astype(int)
champignons.head()

Edible         Color        Shape      Surface  Polypore  Convex  Bell  \
0     2.0          Pale     Polypore       Smooth         1       0     0   
1     0.0         White       Convex       Smooth         0       1     0   
2     0.0  White-Yellow       Convex       Smooth         0       1     0   
3     0.0         White  Bell-shaped       Smooth         0       0     1   
4     0.0         Brown  Bell-shaped  Flat Scales         0       0     1   

   shaped  Depressed  CupFungi  CoralFungi  Conical  Flat  JellyFungi  \
0       0          0         0           0        0     0           0   
1       0          0         0           0        0     0           0   
2       0          0         0           0        0     0           0   
3       1          0         0           0        0     0           0   
4       1          0         0           0        0     0           0   

   Stinkhorns  Earthstars  Puffballs  Corticioid  Chanterelles  Funnel  
0           0           0          0           0             0       0  
1           0           0          0           0             0       0  
2           0           0          0           0             0       0  
3           0           0          0           0             0       0  
4           0           0          0           0             0       0

Question 13 : la meme chose mais pour la colonne surface en factorisant le code 

In [15]:
def colonneIndex(df, column_name):
    v = pd.unique(df[column_name].str.split("-").explode().dropna())
    for x in v:
        df[x] = df[column_name].str.contains(x, na=False).astype(int)
    return df
colonneIndex(champignons, "Surface")

champignons.head()

Edible         Color        Shape      Surface  Polypore  Convex  Bell  \
0     2.0          Pale     Polypore       Smooth         1       0     0   
1     0.0         White       Convex       Smooth         0       1     0   
2     0.0  White-Yellow       Convex       Smooth         0       1     0   
3     0.0         White  Bell-shaped       Smooth         0       0     1   
4     0.0         Brown  Bell-shaped  Flat Scales         0       0     1   

   shaped  Depressed  CupFungi  ...  Funnel  Smooth  Flat Scales  Fibrous  \
0       0          0         0  ...       0       1            0        0   
1       0          0         0  ...       0       1            0        0   
2       0          0         0  ...       0       1            0        0   
3       1          0         0  ...       0       1            0        0   
4       1          0         0  ...       0       0            1        0   

   Patches  Raised Scales  Hairy  Powder  Silky  Velvety  
0        0              0      0       0      0        0  
1        0              0      0       0      0        0  
2        0              0      0       0      0        0  
3        0              0      0       0      0        0  
4        0              0      0       0      0        0  

[5 rows x 29 columns]

Supprimer les colonnes "Shape" et "Surface"

In [17]:
champignons.drop(columns=["Shape", "Surface"], axis=1,inplace=True)
# ici j'ai executé deux fois : resultat dans le deuxiem fois les deux colonnes n'existait plus donc erreur mais le resultat de la question c'et fait 

In [18]:
# verification de la supprission
print("Dimensions de la DataFrame après suppression des colonnes Shape et Surface :", champignons.shape)
champignons.head()

Dimensions de la DataFrame après suppression des colonnes Shape et Surface : (210, 27)


Edible         Color  Polypore  Convex  Bell  shaped  Depressed  CupFungi  \
0     2.0          Pale         1       0     0       0          0         0   
1     0.0         White         0       1     0       0          0         0   
2     0.0  White-Yellow         0       1     0       0          0         0   
3     0.0         White         0       0     1       1          0         0   
4     0.0         Brown         0       0     1       1          0         0   

   CoralFungi  Conical  ...  Funnel  Smooth  Flat Scales  Fibrous  Patches  \
0           0        0  ...       0       1            0        0        0   
1           0        0  ...       0       1            0        0        0   
2           0        0  ...       0       1            0        0        0   
3           0        0  ...       0       1            0        0        0   
4           0        0  ...       0       0            1        0        0   

   Raised Scales  Hairy  Powder  Silky  Velvety  
0              0      0       0      0        0  
1              0      0       0      0        0  
2              0      0       0      0        0  
3              0      0       0      0        0  
4              0      0       0      0        0  

[5 rows x 27 columns]

Question 14 : la liste des couleurs individuelles présentes dans notre jeu de données.

In [32]:
couleurs = pd.unique(champignons["Color"].str.split("-").explode().dropna())

print("Nombre de couleurs :", len(couleurs))
print(couleurs)

Nombre de couleurs : 15
['Pale' 'White' 'Yellow' 'Brown' 'Pink' 'Purple' 'Tan' 'Orange' 'Gray'
 'Red' 'Dark' 'Green' 'Blue' 'Violet' 'Lilac']


Question 15 : Créer un DataFrame

In [33]:
data = {
    "Color": couleurs,
    "R": [255, 255, 255, 165, 255, 128, 210, 255, 128, 255, 0, 0, 0, 238, 200], 
    "G": [229, 255, 255, 42, 192, 0, 180, 165, 128, 0, 0, 128, 0, 130, 162],  
    "B": [180, 255, 0, 42, 203, 128, 140, 0, 128, 0, 0, 0, 255, 238, 200] 
}
rgb = pd.DataFrame(data)
rgb

Color    R    G    B
0     Pale  255  229  180
1    White  255  255  255
2   Yellow  255  255    0
3    Brown  165   42   42
4     Pink  255  192  203
5   Purple  128    0  128
6      Tan  210  180  140
7   Orange  255  165    0
8     Gray  128  128  128
9      Red  255    0    0
10    Dark    0    0    0
11   Green    0  128    0
12    Blue    0    0  255
13  Violet  238  130  238
14   Lilac  200  162  200

Question 16 : Construire un DataFrame colors ayant une seule colonne, qui contient le nom de chaque combinaison unique de couleurs apparaissant parmi les champignons.

In [34]:
couleursUnique= pd.unique(champignons['Color'])
c = pd.DataFrame(couleursUnique,columns=['Color'])
c

Color
0            Pale
1           White
2    White-Yellow
3           Brown
4             NaN
5      White-Pale
6     White-Brown
7      Pink-Brown
8    Purple-Brown
9          Yellow
10     Pale-Brown
11       Pale-Tan
12         Orange
13         Purple
14     Brown-Gray
15     Pale-White
16            Red
17     Gray-Brown
18           Gray
19   Orange-Brown
20   Yellow-Brown
21   Brown-Yellow
22      Gray-Dark
23    Brown-White
24     Brown-Dark
25     Dark-Brown
26           Dark
27      Red-Brown
28     White-Gray
29      Gray-Pale
30       Pink-Red
31  Orange-Yellow
32  Yellow-Orange
33      White-Tan
34     Orange-Red
35     Green-Gray
36     Blue-Green
37           Blue
38     Brown-Pale
39   Yellow-Green
40  Purple-Violet
41   Lilac-Yellow
42       Tan-Pale
43  Violet-Purple
44      Pale-Gray
45         Violet

Question 17 : 


In [36]:
combinaison = c['Color'].str.split('-', expand=True)
combinaison.columns = ['Color1', 'Color2']

for col in ['Color1', 'Color2']:
    combinaison = combinaison.merge(rgb, left_on=col, right_on='Color', how='left', suffixes=('', '_'+col))

combinaison['R'] = combinaison[['R', 'R_Color2']].mean(axis=1)
combinaison['G'] = combinaison[['G', 'G_Color2']].mean(axis=1)
combinaison['B'] = combinaison[['B', 'B_Color2']].mean(axis=1)

combinaison.drop(['Color', 'R_Color2', 'G_Color2', 'B_Color2', 'Color1', 'Color2','Color_Color2'], axis=1, inplace=True)
couleurs = c.join(combinaison, how='left')
couleurs

Color      R      G      B
0            Pale  255.0  229.0  180.0
1           White  255.0  255.0  255.0
2    White-Yellow  255.0  255.0  127.5
3           Brown  165.0   42.0   42.0
4             NaN    NaN    NaN    NaN
5      White-Pale  255.0  242.0  217.5
6     White-Brown  210.0  148.5  148.5
7      Pink-Brown  210.0  117.0  122.5
8    Purple-Brown  146.5   21.0   85.0
9          Yellow  255.0  255.0    0.0
10     Pale-Brown  210.0  135.5  111.0
11       Pale-Tan  232.5  204.5  160.0
12         Orange  255.0  165.0    0.0
13         Purple  128.0    0.0  128.0
14     Brown-Gray  146.5   85.0   85.0
15     Pale-White  255.0  242.0  217.5
16            Red  255.0    0.0    0.0
17     Gray-Brown  146.5   85.0   85.0
18           Gray  128.0  128.0  128.0
19   Orange-Brown  210.0  103.5   21.0
20   Yellow-Brown  210.0  148.5   21.0
21   Brown-Yellow  210.0  148.5   21.0
22      Gray-Dark   64.0   64.0   64.0
23    Brown-White  210.0  148.5  148.5
24     Brown-Dark   82.5   21.0   21.0
25     Dark-Brown   82.5   21.0   21.0
26           Dark    0.0    0.0    0.0
27      Red-Brown  210.0   21.0   21.0
28     White-Gray  191.5  191.5  191.5
29      Gray-Pale  191.5  178.5  154.0
30       Pink-Red  255.0   96.0  101.5
31  Orange-Yellow  255.0  210.0    0.0
32  Yellow-Orange  255.0  210.0    0.0
33      White-Tan  232.5  217.5  197.5
34     Orange-Red  255.0   82.5    0.0
35     Green-Gray   64.0  128.0   64.0
36     Blue-Green    0.0   64.0  127.5
37           Blue    0.0    0.0  255.0
38     Brown-Pale  210.0  135.5  111.0
39   Yellow-Green  127.5  191.5    0.0
40  Purple-Violet  183.0   65.0  183.0
41   Lilac-Yellow  227.5  208.5  100.0
42       Tan-Pale  232.5  204.5  160.0
43  Violet-Purple  183.0   65.0  183.0
44      Pale-Gray  191.5  178.5  154.0
45         Violet  238.0  130.0  238.0

Question 18 : on va crée une  copie pour éviter de modifier les donnée d'origine 
comme à la question 13 

In [37]:
champignons_copie = champignons.copy()
champignons_copie = pd.merge(champignons, rgb, left_on='Color', right_on='Color', how='left')
champignons_copie[['R', 'G', 'B']] = champignons_copie[['R', 'G', 'B']].fillna(-255)
champignons_copie.drop(columns=['Color'], inplace=True)
champignons_copie.head()

Edible        Shape      Surface      R      G      B
0     2.0     Polypore       Smooth  255.0  229.0  180.0
1     0.0       Convex       Smooth  255.0  255.0  255.0
2     0.0       Convex       Smooth -255.0 -255.0 -255.0
3     0.0  Bell-shaped       Smooth  255.0  255.0  255.0
4     0.0  Bell-shaped  Flat Scales  165.0   42.0   42.0

Question 19 : Séparer les données en un jeu d’entraînement et un jeu de test, sur un base 75%/25%.

In [24]:
X = champignons_copie.drop(columns=['Edible'])  # features
y = champignons_copie['Edible']  # cible

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print("Dimensions du jeu d'entraînement X :", X_train.shape)
print("Dimensions du jeu de test X :", X_test.shape)
print("Dimensions du jeu d'entraînement y :", y_train.shape)
print("Dimensions du jeu de test y :", y_test.shape)

Dimensions du jeu d'entraînement X : (157, 28)
Dimensions du jeu de test X : (53, 28)
Dimensions du jeu d'entraînement y : (157,)
Dimensions du jeu de test y : (53,)


Question 20 Entraîner un modèle SVC sur le premier jeu.
Évaluer l’accuracy_score de ce modèle sur le jeu de test, puis détailler les résultats via une
confusion_matrix.

In [38]:
svc_model = SVC()
svc_model.fit(X_train, y_train)

y_pred = svc_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Score:", accuracy)

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
conf_matrix

Accuracy Score: 0.6037735849056604
Confusion Matrix:


array([[28,  0,  1],
       [ 3,  0,  1],
       [16,  0,  4]], dtype=int64)

Question 21 : Reprendre la Question 20, en standardisant les colonnes dans un premier temps, via
un StandardScaler.

In [26]:
standard = StandardScaler()
X_train_standard = standard.fit_transform(X_train)
X_test_standard = standard.transform(X_test)

svc_model2 = SVC()
svc_model2.fit(X_train_standard, y_train)

y_pred = svc_model2.predict(X_test_standard)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Score:", accuracy)

conf_matrix2 = confusion_matrix(y_test, y_pred)
print("Confusion Matrix 2:")
print(conf_matrix2)


Accuracy Score: 0.660377358490566
Confusion Matrix 2:
[[27  0  2]
 [ 2  0  2]
 [11  1  8]]


Question 22 : Reprendre les Questions 20 et 21 en entraînant cette fois un arbre de décision

In [27]:
arbre_decision = DecisionTreeClassifier(max_depth=3, random_state=42)
arbre_decision.fit(X_train, y_train)

# Prédiction sur le jeu de test
Y_pred_dt = arbre_decision.predict(X_test)

# Calcul et affichage de l'accuracy
accuracy_dt = accuracy_score(y_test, Y_pred_dt)
print("Accuracy du modèle d'arbre de décision: " , accuracy_dt)

conf_matrix3 = confusion_matrix(y_test, Y_pred_dt)
print("Confusion Matrix 3:")
print(conf_matrix3)


Accuracy du modèle d'arbre de décision:  0.6037735849056604
Confusion Matrix 3:
[[29  0  0]
 [ 2  1  1]
 [16  2  2]]


Question 23 : 

In [49]:
donnee = export_graphviz(arbre_decision, out_file=None, 
                           feature_names=X_train.columns, 
                           filled=True, rounded=True,
                           special_characters=True)

graph = graphviz.Source(donnee)
print(graph)


digraph Tree {
node [shape=box, style="filled, rounded", color="black", fontname="helvetica"] ;
edge [fontname="helvetica"] ;
0 [label=<Powder &le; 0.5<br/>gini = 0.623<br/>samples = 157<br/>value = [2, 71, 23, 61]>, fillcolor="#ecfcea"] ;
1 [label=<CupFungi &le; 0.5<br/>gini = 0.617<br/>samples = 155<br/>value = [2, 71, 21, 61]>, fillcolor="#ebfcea"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label=<Polypore &le; 0.5<br/>gini = 0.601<br/>samples = 142<br/>value = [2, 69, 16, 55]>, fillcolor="#e1fbdf"] ;
1 -> 2 ;
3 [label=<gini = 0.584<br/>samples = 135<br/>value = [2, 68, 12, 53]>, fillcolor="#ddfadb"] ;
2 -> 3 ;
4 [label=<gini = 0.571<br/>samples = 7<br/>value = [0, 1, 4, 2]>, fillcolor="#b0d8f5"] ;
2 -> 4 ;
5 [label=<G &le; 210.0<br/>gini = 0.615<br/>samples = 13<br/>value = [0, 2, 5, 6]>, fillcolor="#fae6fc"] ;
1 -> 5 ;
6 [label=<gini = 0.595<br/>samples = 11<br/>value = [0, 2, 3, 6]>, fillcolor="#f0b5f5"] ;
5 -> 6 ;
7 [label=<gini = 0.0<br/>samples = 2<br/>

Question 24 : 

In [47]:
dump(svc_model2, 'svc_model.joblib')
dump(arbre_decision, 'arbre_décision.joblib')

['arbre_décision.joblib']

In [45]:


def load_model(model_name):
    if model_name == "svm":
        return load("svc_model.joblib")
    elif model_name == "DecisionTree":
        return load("arbre_décision.joblib")
    else:
        raise ValueError("Model not found")

def predict(model, r, g, b, shapes, surfaces):
    shapes = shapes.split(",") if shapes else []
    surfaces = surfaces.split(",") if surfaces else []

    shapes_name = ['Bellshaped','FalseMorels','Flat', 'Funnelshaped','JellyFungi','Knobbed',  'Polypore',
              'Bolete','Cauliflower','Chanterelles','Conical','Convex','CoralFungi','Corticioid','CupFungi',
              'Cylindrical','Depressed','Earthstars','Puffballs','Shellshaped', 'Stinkhorns','ToothFungi','TrueMorels','Truffles']

    surfaces_name = ['Fibrous','FlatScales','Hairy','Patches','Powder','RaisedScales','Silky','Smooth','Velvety']

    shapes_vector = np.zeros(len(shapes))
    surfaces_vector = np.zeros(len(surfaces))

    for shape in shapes:
        if shape in shapes_name:
            shapes_vector[shapes_name.index(shape)] = 1

    for surface in surfaces:
        if surface in surfaces_name:
            surfaces_vector[surfaces_name.index(surface)] = 1

    features = np.concatenate([np.array([r, g, b], dtype=float), shapes_vector, surfaces_vector]).reshape(1, -1)
    prediction = model.predict(features)
    return "Edible" if prediction == 0 else "Inedible" if prediction == 1 else "Poisonous"

if __name__ == "__main__":   
    r, g, b, model_name = sys.argv[1:5]
    shapes = sys.argv[5].split(",")
    surfaces = sys.argv[6].split(",")
    model = load_model(model_name)
    result = predict(model, int(r), int(g), int(b), shapes, surfaces)
    print(result)


Output: 
